# Converting XML files into CSV files

Our dataset has been provided to us in XML format, we will first write a function to convert these XML files into CSV files

In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format 

In [2]:
def xml_to_csv(train_bool, test_bool, year, patient_nb, variable):
    # Open the patient's XML train or XML test file and parse it with ElementTree
    if train_bool:
        tree = ET.parse(f"data/xml/{year}/train/{patient_nb}-ws-training.xml")
    elif test_bool:
        tree = ET.parse(f"data/xml/{year}/test/{patient_nb}-ws-testing.xml")
    else:
        raise ValueError("You must specify either train_bool or test_bool as True.")
    
    root = tree.getroot()

    # Open the CSV file in write mode and create a csv.writer object
    if train_bool:
        csv_filename = f"data/csv/{year}/train/{patient_nb}-ws-training-{variable}.csv"
    elif test_bool:
        csv_filename = f"data/csv/{year}/test/{patient_nb}-ws-testing-{variable}.csv"
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)

        if variable == "glucose_level":
            # Write the column headers in the CSV file
            writer.writerow(['ts', 'value'])

            # Browse the elements of the XML file and write the data to the CSV file
            for child in root[0]:
                row = []
                row.append(child.attrib['ts'])
                row.append(child.attrib['value'])
                writer.writerow(row)

        elif variable == "basis_heart_rate":
            # Write the column headers in the CSV file
            writer.writerow(['ts', 'value'])

            # Browse the elements of the XML file and write the data to the CSV file
            for child in root[12]:
                row = []
                row.append(child.attrib['ts'])
                row.append(child.attrib['value'])
                writer.writerow(row)

        elif variable == "basal":
            # Write the column headers in the CSV file
            writer.writerow(['ts', 'value'])

            # Browse the elements of the XML file and write the data to the CSV file
            for child in root[2]:
                row = []
                row.append(child.attrib['ts'])
                row.append(child.attrib['value'])
                writer.writerow(row)

        elif variable == "bolus":
            # Write the column headers in the CSV file
            writer.writerow(['ts_begin', 'ts_end', 'type', 'dose', 'bwz_carb_input'])

            # Browse the elements of the XML file and write the data to the CSV file
            for child in root[4]:
                row = []
                row.append(child.attrib['ts_begin'])
                row.append(child.attrib['ts_end'])
                row.append(child.attrib['type'])
                row.append(child.attrib['dose'])
                row.append(child.attrib['bwz_carb_input'])
                writer.writerow(row)

        elif variable == "exercise":
            # Write the column headers in the CSV file
            writer.writerow(['ts', 'intensity', 'type', 'duration', 'competitive'])

            # Browse the elements of the XML file and write the data to the CSV file
            for child in root[11]:
                row = []
                row.append(child.attrib['ts'])
                row.append(child.attrib['intensity'])
                row.append(child.attrib['type'])
                row.append(child.attrib['duration'])
                row.append(child.attrib['competitive'])
                writer.writerow(row)

        else:
            raise ValueError("You must specify either a variable.")

# Building the Multivariate Dataset

For the different patients in the study, we had access to many variables (see dataset description: http://smarthealth.cs.ohio.edu/OhioT1DM-dataset.html). We made the arbitrary choice to keep only some variables for our model (glucose leve, basal, bolus, exercise, basis heart rate). We create here a function that takes care of pre-processing the data and creating the multivariate dataset that will allow us to implement our model.

In [5]:
# we create all the csv files that will be useful
xml_to_csv(True, False, "2018", "559", "glucose_level")
xml_to_csv(False, True, "2018", "559", "glucose_level")

xml_to_csv(True, False, "2018", "559", "basis_heart_rate")
xml_to_csv(False, True, "2018", "559", "basis_heart_rate")

xml_to_csv(True, False, "2018", "559", "basal")
xml_to_csv(False, True, "2018", "559", "basal")

xml_to_csv(True, False, "2018", "559", "bolus")
xml_to_csv(False, True, "2018", "559", "bolus")

xml_to_csv(True, False, "2018", "559", "exercise")
xml_to_csv(False, True, "2018", "559", "exercise")

FileNotFoundError: [Errno 2] No such file or directory: 'data/xml/2018/train/559-ws-training.xml'

In [ ]:
# we import our dataframes
bg_train_df = pd.read_csv('data/csv/2018/train/559-ws-training-glucose_level.csv')
bg_test_df = pd.read_csv('data/csv/2018/test/559-ws-testing-glucose_level.csv')

bhr_train_df = pd.read_csv('data/csv/2018/train/559-ws-training-basis_heart_rate.csv')
bhr_test_df = pd.read_csv('data/csv/2018/test/559-ws-testing-basis_heart_rate.csv')

bol_train_df = pd.read_csv('data/csv/2018/train/559-ws-training-bolus.csv')
bol_test_df = pd.read_csv('data/csv/2018/test/559-ws-testing-bolus.csv')

bas_train_df = pd.read_csv('data/csv/2018/train/559-ws-training-basal.csv')
bas_test_df = pd.read_csv('data/csv/2018/test/559-ws-testing-basal.csv')

exo_train_df = pd.read_csv('data/csv/2018/train/559-ws-training-exercise.csv')
exo_test_df = pd.read_csv('data/csv/2018/test/559-ws-testing-exercise.csv')

In [ ]:
def build_multivariate_datasets(bg_train_df, bg_test_df,
                                bhr_train_df, bhr_test_df,
                                bas_train_df, bas_test_df,
                                bol_train_df, bol_test_df,
                                exo_train_df, exo_test_df):
    
    # on convertit toutes les colonnes 'timestamp' en colonnes de type datetime
    bg_train_df['ts'] = pd.to_datetime(bg_train_df['ts'], format = "%d-%m-%Y %H:%M:%S")
    bg_test_df['ts'] = pd.to_datetime(bg_test_df['ts'], format = "%d-%m-%Y %H:%M:%S")

    bhr_train_df['ts'] = pd.to_datetime(bhr_train_df['ts'], format = "%d-%m-%Y %H:%M:%S")
    bhr_test_df['ts'] = pd.to_datetime(bhr_test_df['ts'], format = "%d-%m-%Y %H:%M:%S")

    bol_train_df['ts_begin'] = pd.to_datetime(bol_train_df['ts_begin'], format = "%d-%m-%Y %H:%M:%S")
    bol_train_df['ts_end'] = pd.to_datetime(bol_train_df['ts_end'], format = "%d-%m-%Y %H:%M:%S")
    bol_test_df['ts_begin'] = pd.to_datetime(bol_test_df['ts_begin'], format = "%d-%m-%Y %H:%M:%S")
    bol_test_df['ts_end'] = pd.to_datetime(bol_test_df['ts_end'], format = "%d-%m-%Y %H:%M:%S")

    bas_train_df['ts'] = pd.to_datetime(bas_train_df['ts'], format = "%d-%m-%Y %H:%M:%S")
    bas_test_df['ts'] = pd.to_datetime(bas_test_df['ts'], format = "%d-%m-%Y %H:%M:%S")

    exo_train_df['ts'] = pd.to_datetime(exo_train_df['ts'], format = "%d-%m-%Y %H:%M:%S")
    exo_test_df['ts'] = pd.to_datetime(exo_test_df['ts'], format = "%d-%m-%Y %H:%M:%S")



    # on supprime les colonnes inutiles
    bol_train_df = bol_train_df.drop('ts_end', axis=1) # car ts_end identique à ts_begin
    bol_test_df = bol_test_df.drop('ts_end', axis=1)
    bol_train_df = bol_train_df.drop('type', axis=1) # car 'type' ne change pas de valeur
    bol_test_df = bol_test_df.drop('type', axis=1)

    exo_train_df = exo_train_df.drop('type', axis=1) # car 'type' vide
    exo_test_df = exo_test_df.drop('type', axis=1)
    exo_train_df = exo_train_df.drop('competitive', axis=1) # car NA
    exo_test_df = exo_test_df.drop('competitive', axis=1)

    ### regrouper les dataframes bg et bhr

    ## faire correspondre les dates de débuts et de fin des dataframes

    # train
    # date début bg_train_df = 2021-12-07 01:17:00
    # date début bhr_train_df = 2021-12-07 12:57:00
    bg_train_df = bg_train_df[bg_train_df['ts'] > '2021-12-07 12:52:00']

    # date fin bg_train_df = 2022-01-17 23:56:00
    # date fin bhr_train_df = 2022-01-17 23:55:00

    # test
    # date début bg_test_df = 2022-01-18 00:01:00	
    # date début bhr_test_df = 2022-01-18 00:00:00

    # date fin bg_test_df = 2022-01-27 23:38:00
    # date fin bhr_test_df = 2022-01-27 17:55:00
    bg_test_df = bg_test_df[bg_test_df['ts'] < '2022-01-27 17:55:00']

    ## fusion des deux datasets

    # train
    bg_train_df = bg_train_df.set_index('ts')
    bhr_train_df = bhr_train_df.set_index('ts')

    bg_train_df = bg_train_df.rename(columns = {'value': 'value_bg'})
    bhr_train_df = bhr_train_df.rename(columns = {'value': 'value_bhr'})

    merged_train_df = bg_train_df.join(bhr_train_df, how='outer')

    # test
    bg_test_df = bg_test_df.set_index('ts')
    bhr_test_df = bhr_test_df.set_index('ts')

    bg_test_df = bg_test_df.rename(columns = {'value': 'value_bg'})
    bhr_test_df = bhr_test_df.rename(columns = {'value': 'value_bhr'})

    merged_test_df = bg_test_df.join(bhr_test_df, how='outer')

    ## imputation des valeurs manquantes

    # train
    merged_train_df['value_bg'] = merged_train_df['value_bg'].interpolate(method='time')
    merged_train_df['value_bhr'] = merged_train_df['value_bhr'].interpolate(method='time')

    # test
    merged_test_df.loc['2022-01-18 00:00:00', 'value_bg'] = 179

    merged_test_df['value_bg'] = merged_test_df['value_bg'].interpolate(method='time')
    merged_test_df['value_bhr'] = merged_test_df['value_bhr'].interpolate(method='time')



    ### regrouper les dataframes bg, bhr et bas

    ## faire correspondre les dates de débuts et de fin des dataframes
    bas_train_df = bas_train_df[bas_train_df['ts'] > '2021-12-08 00:00:00'] # la date de début doit être postérieure à la date de début du df merged_train_df

    ## fusion des deux datasets
    bas_train_df = bas_train_df.set_index('ts')
    bas_train_df = bas_train_df.rename(columns = {'value': 'value_bas'})

    bas_test_df = bas_test_df.set_index('ts')
    bas_test_df = bas_test_df.rename(columns = {'value': 'value_bas'})

    merged_train_df = merged_train_df.join(bas_train_df, how='outer')
    merged_test_df = merged_test_df.join(bas_test_df, how='outer')

    ## imputation des valeurs manquantes
    merged_train_df['value_bg'] = merged_train_df['value_bg'].interpolate(method='time')
    merged_train_df['value_bhr'] = merged_train_df['value_bhr'].interpolate(method='time')

    merged_test_df['value_bg'] = merged_test_df['value_bg'].interpolate(method='time')
    merged_test_df['value_bhr'] = merged_test_df['value_bhr'].interpolate(method='time')

    merged_train_df['value_bas'] = merged_train_df['value_bas'].fillna(0)
    merged_test_df['value_bas'] = merged_test_df['value_bas'].fillna(0)



    ### regrouper les dataframes bg, bhr, bas et exo
    exo_train_df = exo_train_df.set_index('ts')
    exo_test_df = exo_test_df.set_index('ts')

    exo_train_df = exo_train_df.rename(columns = {'intensity': 'intensity_exo', 'duration': 'duration_exo'})
    exo_test_df = exo_test_df.rename(columns = {'intensity': 'intensity_exo', 'duration': 'duration_exo'})


    merged_train_df = merged_train_df.join(exo_train_df, how='outer')
    merged_test_df = merged_test_df.join(exo_test_df, how='outer')


    merged_train_df['value_bg'] = merged_train_df['value_bg'].interpolate(method='time')
    merged_train_df['value_bhr'] = merged_train_df['value_bhr'].interpolate(method='time')

    merged_test_df['value_bg'] = merged_test_df['value_bg'].interpolate(method='time')
    merged_test_df['value_bhr'] = merged_test_df['value_bhr'].interpolate(method='time')


    merged_train_df['value_bas'] = merged_train_df['value_bas'].fillna(0)
    merged_train_df['intensity_exo'] = merged_train_df['intensity_exo'].fillna(0)
    merged_train_df['duration_exo'] = merged_train_df['duration_exo'].fillna(0)

    merged_test_df['value_bas'] = merged_test_df['value_bas'].fillna(0)
    merged_test_df['intensity_exo'] = merged_test_df['intensity_exo'].fillna(0)
    merged_test_df['duration_exo'] = merged_test_df['duration_exo'].fillna(0)



    ### regrouper les dataframes bg, bhr, bas, exo et bolus
    bol_train_df.rename(columns={"ts_begin": "ts"}, inplace=True)
    bol_test_df.rename(columns={"ts_begin": "ts"}, inplace=True)

    bol_train_df = bol_train_df[bol_train_df['ts'] > '2021-12-07 12:57:00']
    bol_test_df = bol_test_df[bol_test_df['ts'] < '2022-01-27 17:55:00']

    bol_train_df = bol_train_df.set_index('ts')
    bol_test_df = bol_test_df.set_index('ts')


    bol_train_df = bol_train_df.rename(columns = {'dose': 'dose_bolus', 'bwz_carb_input': 'bolus_carbs_input'})
    bol_test_df = bol_test_df.rename(columns = {'dose': 'dose_bolus', 'bwz_carb_input': 'bolus_carbs_input'})

    merged_train_df = merged_train_df.join(bol_train_df, how='outer')
    merged_test_df = merged_test_df.join(bol_test_df, how='outer')

    merged_train_df['value_bg'] = merged_train_df['value_bg'].interpolate(method='time')
    merged_train_df['value_bhr'] = merged_train_df['value_bhr'].interpolate(method='time')

    merged_test_df['value_bg'] = merged_test_df['value_bg'].interpolate(method='time')
    merged_test_df['value_bhr'] = merged_test_df['value_bhr'].interpolate(method='time')


    merged_train_df['value_bas'] = merged_train_df['value_bas'].fillna(0)
    merged_train_df['intensity_exo'] = merged_train_df['intensity_exo'].fillna(0)
    merged_train_df['duration_exo'] = merged_train_df['duration_exo'].fillna(0)
    merged_train_df['dose_bolus'] = merged_train_df['dose_bolus'].fillna(0)
    merged_train_df['bolus_carbs_input'] = merged_train_df['bolus_carbs_input'].fillna(0)

    merged_test_df['value_bas'] = merged_test_df['value_bas'].fillna(0)
    merged_test_df['intensity_exo'] = merged_test_df['intensity_exo'].fillna(0)
    merged_test_df['duration_exo'] = merged_test_df['duration_exo'].fillna(0)
    merged_test_df['dose_bolus'] = merged_test_df['dose_bolus'].fillna(0)
    merged_test_df['bolus_carbs_input'] = merged_test_df['bolus_carbs_input'].fillna(0)

    return merged_train_df, merged_test_df        